# PRAC 2. Visualizacion de datos
En este notebook desarrollaremos la preparacion de datos necesaria para la creacion del dashboard publicado en Tableau Public: 

----
In this notebook, we will develop the data preparation necessary for creating the dashboard published in Tableau Public:

## Data loading
El data set a trabajar será una composición de múltiples data sets extraídos de la página web https://ourworldindata.org, datos que giran en torno a la categoría de Energía y Medio Ambiente. Los archivos estan disponibles inicialmente como archivos de tipo ZIP, por lo cual es necesario extraer los archivos CSV y asignarlas a data frames que luego utilizaremos para realizar la limpieza de datos 

---
The data set to be worked on will be a composition of multiple data sets extracted from the website https://ourworldindata.org, data that revolves around the category of Energy and Environment. The files are initially available as ZIP files, so it is necessary to extract the CSV files and assign them to data frames that we will then use to perform data cleaning.

In [4]:
# Data load to data frames
import zipfile
import pandas as pd
import os

def load_all_zips_from_folder(folder_path):
    """
    Scans a folder for ZIP files and loads every internal CSV into a dictionary.
    """
    all_data_frames = {}

    # 1. Iterate through all files in the provided folder
    for item in os.listdir(folder_path):
        if item.endswith('.zip'):
            zip_path = os.path.join(folder_path, item)
            print(f"--- Processing Archive: {item} ---")

            try:
                # 2. Open the current zip file
                with zipfile.ZipFile(zip_path, 'r') as z:
                    for file_name in z.namelist():
                        # 3. Filter for CSV files
                        if file_name.endswith('.csv'):
                            with z.open(file_name) as f:
                                # Create a unique key: "archive_name/file_name"
                                unique_key = f"{item}/{file_name}"
                                
                                # Load into DataFrame
                                df = pd.read_csv(f)
                                all_data_frames[unique_key] = df
                                print(f" Loaded: {file_name}")
                                
            except Exception as e:
                print(f" Could not read {item}: {e}")

    return all_data_frames

# --- Implementation ---
path = 'data_original' 
data_library = load_all_zips_from_folder(path)
print(f"\nTotal CSVs loaded: {len(data_library)}")

--- Processing Archive: forest-area-km.zip ---
 Loaded: forest-area-km.csv
--- Processing Archive: improved-sanitation-facilities-vs-gdp-per-capita.zip ---
 Loaded: improved-sanitation-facilities-vs-gdp-per-capita.csv
--- Processing Archive: co-emissions-per-capita.zip ---
 Loaded: co-emissions-per-capita.csv
--- Processing Archive: fossil-fuel-consumption-by-fuel-type.zip ---
 Loaded: fossil-fuel-consumption-by-fuel-type.csv
--- Processing Archive: plastic-waste-mismanaged.zip ---
 Loaded: plastic-waste-mismanaged.csv
--- Processing Archive: stratospheric-ozone-concentration.zip ---
 Loaded: stratospheric-ozone-concentration.csv
--- Processing Archive: share-without-improved-sanitation.zip ---
 Loaded: share-without-improved-sanitation.csv
--- Processing Archive: land-use-over-the-long-term.zip ---
 Loaded: land-use-over-the-long-term.csv
--- Processing Archive: number-without-safe-drinking-water.zip ---
 Loaded: number-without-safe-drinking-water.csv
--- Processing Archive: annual-ar

In [7]:
# Visual check of the headers of each data frame in the new data dictionary

def display_headers(data_dict, rows=5):
    """
    Iterates through the dictionary and prints the header of each DataFrame.
    """

    for identifier, df in data_dict.items():
        print("\n" + "="*50)
        print(f"FILE: {identifier}")
        print("="*50)
        
        # Display the first rows
        print(df.head(rows))
        print(f"\nShape: {df.shape} (Rows, Columns)")

# --- Implementation ---
display_headers(data_library, rows=3)


FILE: forest-area-km.zip/forest-area-km.csv
        Entity Code  Year  Forest area
0  Afghanistan  AFG  1990    1208440.0
1  Afghanistan  AFG  1991    1208440.0
2  Afghanistan  AFG  1992    1208440.0

Shape: (7846, 4) (Rows, Columns)

FILE: improved-sanitation-facilities-vs-gdp-per-capita.zip/improved-sanitation-facilities-vs-gdp-per-capita.csv
        Entity Code  Year  \
0  Afghanistan  AFG  2000   
1  Afghanistan  AFG  2001   
2  Afghanistan  AFG  2002   

   Share of the population using improved sanitation facilities  \
0                                          26.466162              
1                                          26.488068              
2                                          28.414984              

   GDP per capita, PPP (constant 2021 international $)  \
0                                          1617.8264     
1                                          1454.1108     
2                                          1774.3087     

  World regions according to OWID

## Analisis inicial / Initial analysis
Encontramos que aunque toda la data cuenta con las mismas llaves: Entity, Code, Year. No todos tienen el mismo nivel de detalle ni los mismos rangos de tiempo disponibles para cada variable. Por lo que hay que tener presente los siguientes puntos antes de realizar cualquier analisis:
1. Solo los paises tienen asignado un valor en la variable "Code"
2. Los valores que no tienen "Code" asignado corresponden a agregaciones como continente, hemisferio o total mundo.
3. No todos los paises tienen data en todas las variables
4. El rango de años disponibles varia en cada variable.

Teniendo esto presente procederemos a unir todos los datos en un solo archivo, usando como llaves los valores ya mencionados (Entity, Code, Year). Esto con el fin de facilitar la ingesta de datos por parte de Tableau. Y desde alli, teniendo en cuenta los puntos ya mencionados realizaremos los respectivos analisis visuales.

----
We found that although all data has the same keys: Entity, Code, Year, not all the data has the same level of detail or the same time ranges available for each variable. Therefore, the following points must be kept in mind before performing any analysis:
1. Only countries are assigned a value in the "Code" variable.
2. Values that do not have a "Code" assigned correspond to aggregations such as continent, hemisphere, or world total.
3. Not all countries have data in all variables.
4. The same data is not available for all years.

With this in mind, we will proceed to merge all the data into a single file, using the aforementioned values (Entity, Code, Year) as keys. This is in order to facilitate data ingestion by Tableau from which we will perform the respective visual analysis taking into account the points already mentioned.

In [22]:
def merge_library_on_keys(data_dict, key_cols=['Entity', 'Code', 'Year']):
    """
    Merges all DataFrames in the dictionary into one single DataFrame
    """
    items = list(data_dict.items())

    # 1. Initialize the merged dataframe with the first item
    _, df_merged = items[0]
    df_merged = df_merged.copy()

    # 2. Merge the other items 
    for file_path, df_next in items[1:]:
        df_merged = pd.merge(
            df_merged, 
            df_next, 
            on=key_cols, 
            how='outer'
        )

    return df_merged

# --- Implementation ---
df_merged = merge_library_on_keys(data_library)
df_merged.head()

Final Merged Shape: (45226, 31)


,Entity,Code,Year,Forest area,Share of the population using improved sanitation facilities,"GDP per capita, PPP (constant 2021 international $)",World regions according to OWID,Annual CO₂ emissions (per capita)_x,Gas consumption - TWh,Coal consumption - TWh,...,Fossil fuels per capita (kWh),Concentrations of fine particulate matter (PM2.5) - Residence area type: Total,Fossil fuels (TWh),Global average temperature anomaly relative to 1861-1890,Lower bound of the annual temperature anomaly (95% confidence interval),Upper bound of the annual temperature anomaly (95% confidence interval),Annual CO₂ emissions,Access to electricity (% of population),Annual CO₂ emissions (per capita)_y,Consumption of controlled substance (zero-filled) - Chemical: All (Ozone-depleting)
0,Afghanistan,AFG,1990,1208440.0,NaN,NaN,NaN,0.168054,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2024326.1,NaN,0.168054,NaN
1,Afghanistan,AFG,1991,1208440.0,NaN,NaN,NaN,0.156411,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1914301.0,NaN,0.156411,NaN
2,Afghanistan,AFG,1992,1208440.0,NaN,NaN,NaN,0.111609,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1482054.0,NaN,0.111609,NaN
3,Afghanistan,AFG,1993,1208440.0,NaN,NaN,NaN,0.099506,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1486943.0,NaN,0.099506,NaN
4,Afghanistan,AFG,1994,1208440.0,NaN,NaN,NaN,0.089462,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1453829.0,NaN,0.089462,NaN


In [26]:
df_merged.describe()

,Year,Forest area,Share of the population using improved sanitation facilities,"GDP per capita, PPP (constant 2021 international $)",Annual CO₂ emissions (per capita)_x,Gas consumption - TWh,Coal consumption - TWh,Oil consumption - TWh,Mismanaged plastic waste (metric tons year-1),Minimum daily concentration,...,Fossil fuels per capita (kWh),Concentrations of fine particulate matter (PM2.5) - Residence area type: Total,Fossil fuels (TWh),Global average temperature anomaly relative to 1861-1890,Lower bound of the annual temperature anomaly (95% confidence interval),Upper bound of the annual temperature anomaly (95% confidence interval),Annual CO₂ emissions,Access to electricity (% of population),Annual CO₂ emissions (per capita)_y,Consumption of controlled substance (zero-filled) - Chemical: All (Ozone-depleting)
count,45226.000000,7.846000e+03,5653.000000,7236.000000,26509.000000,6405.000000,6405.000000,6405.000000,1.710000e+02,45.000000,...,5145.000000,2100.000000,6405.000000,528.000000,528.000000,528.000000,2.938400e+04,6912.000000,26509.000000,5.642000e+03
mean,1497.517225,7.483606e+07,81.309421,21729.221482,3.821372,1284.206388,1682.740233,2304.061156,1.085219e+06,115.666667,...,31749.398187,23.129940,5271.007785,0.304426,0.195156,0.413695,4.202270e+08,81.945391,3.821372,5.026595e+03
std,1711.155450,3.163738e+08,24.023910,23785.629715,14.312865,3670.941634,5153.876464,6079.290689,5.838606e+06,29.522719,...,33139.280595,14.527492,14359.423401,0.412579,0.454342,0.376927,1.972092e+09,28.721936,14.312865,4.577049e+04
min,-10000.000000,0.000000e+00,6.727768,510.822800,0.000000,0.000000,0.000000,0.000000,3.000000e+00,73.000000,...,0.000000,4.918790,0.000000,-0.320683,-0.566759,-0.134624,0.000000e+00,0.533899,0.000000,-1.026720e+04
25%,1856.000000,7.206200e+04,69.018860,4372.770950,0.171333,11.880710,4.348958,80.388960,1.903500e+03,97.000000,...,10360.155000,11.659133,142.379650,0.008761,-0.160436,0.162610,3.810560e+05,72.800000,0.171333,2.800000e+00
50%,1951.000000,2.576958e+06,93.954120,12710.775000,1.023368,82.808750,47.364048,204.001390,2.756500e+04,105.000000,...,24974.670000,19.086140,406.549220,0.167542,0.077409,0.303260,5.080756e+06,99.288955,1.023368,2.900000e+01
75%,1993.000000,1.854451e+07,99.337180,32508.406000,4.327494,440.570400,446.305180,1013.221800,2.785035e+05,124.000000,...,40051.790000,31.469703,1870.448500,0.485473,0.425147,0.573123,5.365634e+07,100.000000,4.327494,3.114050e+02
max,2025.000000,4.236433e+09,100.000000,174339.000000,782.743400,41278.270000,45850.543000,55292.082000,6.177370e+07,195.000000,...,318559.700000,72.182890,142420.890000,1.937535,1.884537,2.196952,3.859858e+10,100.000000,782.743400,1.662589e+06


## Data fusionada / Merged data

La data ha sido fusionada en un solo archivo 'merged_data', el cual sera cargado en Tableau para la creacion de los dashboards y el respectivo analisis.

----
The data has been merged into a single file called 'merged_data', which will be loaded into Tableau for the creation of dashboards and the respective analysis.

In [29]:
# Export the data frame to a CSV
import os
df_merged.to_csv('data_cleaned/merged_data.csv', index=False)
print(os.listdir('data_cleaned'))

['merged_data.csv']
